## ChatLab: Chat Experiments, Simplified

💬🔬

In [1]:
import chatlab

chat = chatlab.Chat()

await chat("What kind of bird is a murkrow?")

A Murkrow is a dark-type Pokémon species in the Pokémon franchise. Although it is bird-like in appearance, it is not based on any specific real-world bird species.

In [2]:
from chatlab import Chat, user, system

chat = Chat()

await chat(
    system("You are a data science tutor running inside of a Jupyter "
           "Notebook environment that has the full NumFOCUS/PyData stack."),
    user("Help me work with circulation-grade-6.csv"),
)

Sure, I can help you work with the "circulation-grade-6.csv" file. 

To get started, we first need to import the necessary libraries. Here, we will use `pandas` for data manipulation and analysis:

```python
import pandas as pd
```

Next, we can read the CSV file into a DataFrame using the `read_csv()` function from `pandas`:

```python
df = pd.read_csv('circulation-grade-6.csv')
```

Now that we have loaded the data, we can start exploring and analyzing it.

To get a quick overview of the data, you can use the `head()` method to display the first few rows of the DataFrame:

```python
df.head()
```

To access specific columns in the DataFrame, you can use the column names as keys. For example, to access the "Student ID" and "Circulation Count" columns, you can use:

```python
student_id = df['Student ID']
circulation_count = df['Circulation Count']
```

You can perform basic summary statistics on columns using the `describe()` method. For numerical columns like "Circulation Count", this will provide information such as mean, standard deviation, minimum, maximum, and quartiles:

```python
circulation_count.describe()
```

If you want to filter the data based on certain conditions, you can use Boolean indexing. For example, to select rows where the "Circulation Count" is greater than 10, you can do:

```python
high_circulation = df[df['Circulation Count'] > 10]
```

You can also perform various aggregations and computations on the DataFrame columns. For instance, to calculate the total circulation count, you can use the `sum()` method:

```python
total_circulation = df['Circulation Count'].sum()
```

These are just some basic operations you can perform on the data. Depending on the specific analysis or tasks you have in mind, there may be additional considerations.

Let me know if you need any further assistance or if you have any specific questions about the dataset!

## Introducing Chat Functions

Where `chatlab` (and OpenAI, really) take it next level is with _Chat Functions_. You can declare a function with a schema then register it in your `Chat` for GPT Models to call. You may recall this kind of behavior from [ChatGPT Plugins](https://noteable.io/chatgpt-plugin-for-notebook/). Now, you can take this even further with your own custom code.

As an example, let's give the large language models the ability to tell time.

In [3]:
from datetime import datetime
from pytz import timezone, all_timezones, utc
from typing import Optional
from pydantic import BaseModel

def what_time(tz: Optional[str] = None):
    '''Current time, defaulting to UTC'''
    if tz is None:
        pass
    elif tz in all_timezones:
        tz = timezone(tz)
    else:
        return 'Invalid timezone'
        
    return datetime.now(tz).strftime('%I:%M %p')

class WhatTime(BaseModel):
    tz: Optional[str]

Let's break this down.

`what_time` is the function we're going to provide access to. Its docstring forms the `description` for the model while the schema comes from the pydantic `BaseModel` called `WhatTime`.

In [4]:
import chatlab

chat = chatlab.Chat()

# Register our function
chat.register(what_time, WhatTime)

{'name': 'what_time',
 'description': 'Current time, defaulting to UTC',
 'parameters': {'title': 'WhatTime',
  'type': 'object',
  'properties': {'tz': {'title': 'Tz', 'type': 'string'}}}}

In [5]:
await chat("What time is it?")

The current time is 07:37 AM.

In [6]:
chat.messages

[{'role': 'user', 'content': 'What time is it?'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'what_time', 'arguments': '{}'}},
 {'role': 'function', 'content': '07:37 AM', 'name': 'what_time'},
 {'role': 'assistant', 'content': 'The current time is 07:37 AM.'}]

In [7]:
await chat("Great. What time is it in Osaka?")

The current time in Osaka is 11:37 PM.

Under the hood, `chatlab` is keeping track of the conversation history for you in `.messages`

In [8]:
chat.messages

[{'role': 'user', 'content': 'What time is it?'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'what_time', 'arguments': '{}'}},
 {'role': 'function', 'content': '07:37 AM', 'name': 'what_time'},
 {'role': 'assistant', 'content': 'The current time is 07:37 AM.'},
 {'role': 'user', 'content': 'Great. What time is it in Osaka?'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'what_time',
   'arguments': '{\n  "tz": "Asia/Tokyo"\n}'}},
 {'role': 'function', 'content': '11:37 PM', 'name': 'what_time'},
 {'role': 'assistant', 'content': 'The current time in Osaka is 11:37 PM.'}]

## Outro

Thanks for reading through! Please post issues and reach out. I want to hear your experiences!